In [3]:
# 라이브러리 불러오기
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

# 예제 2.7의 get_financials() 함수를 사용 (Dapada apiKey 필요)

my_key = "----발급받은 API 키를 입력-----"

# 특정 종목(stockCode)의 재무항목(indicatorName) 데이터를 가져오는 함수를 정의
def get_financials(stockCode, indicatorName, apiKey, consolidated=True, ttm=True):
    
    if consolidated:   # 연결 기준
        if ttm:        # Trailing 12 Months (12개월 누적)
            url = f"https://api.dapada.io/company/getConsolidatedFinancialDataByTTM?apiKey={apiKey}&indicatorName={indicatorName}&stockCode={stockCode}"            
        else:          # 해당 분기 기준 (3개월 집계)
            url = f"https://api.dapada.io/company/getConsolidatedFinancialDataByCUR?apiKey={apiKey}&indicatorName={indicatorName}&stockCode={stockCode}"       
            
    else:              # 별도 기준
        if ttm:        # Trailing 12 Months (12개월 누적)
            url = f"https://api.dapada.io/company/getSeparatedFinancialDataByTTM?apiKey={apiKey}&indicatorName={indicatorName}&stockCode={stockCode}"            
        else:          # 해당 분기 기준 (3개월 집계)
            url = f"https://api.dapada.io/company/getSeparatedFinancialDataByCUR?apiKey={apiKey}&indicatorName={indicatorName}&stockCode={stockCode}"       
           
    headers = {"apiKey": f"{apiKey}"}
    response = requests.get(url, headers=headers)
    result = response.json()
    return pd.DataFrame(result)

# 삼성전자(005930)의 연결 재무제표 (분기 기준)에서 '영업이익' 항목의 데이터 수집
df_samsung = get_financials(stockCode='005930', 
                            indicatorName='영업이익', 
                            apiKey=my_key, 
                            consolidated=True, 
                            ttm=False)

# 영업이익의 변화율을 계산
df_samsung['change'] = df_samsung['value'].pct_change()

# quarter 열을 인덱스로 설정
df_samsung = df_samsung.set_index('quarter')

# SK하이닉스(000660)의 연결 재무제표 (분기 기준)에서 '영업이익' 항목의 데이터 수집
df_skhynix = get_financials(stockCode='000660', 
                            indicatorName='영업이익', 
                            apiKey=my_key, 
                            consolidated=True, 
                            ttm=False)

# 영업이익의 변화율을 계산
df_skhynix['change'] = df_skhynix['value'].pct_change()

# quarter 열을 인덱스로 설정
df_skhynix = df_skhynix.set_index('quarter')

df_samsung

,value,change
quarter,,
2023-Q3,2433534000000,NaN
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459
2022-Q3,9389198000000,1.180426
2022-Q2,11098805000000,0.182082


In [4]:
# 데이터 값의 형식을 지정 (소수정 이하 둘째 자리, 천 단위, 소수점 표시)
df_samsung.style.format(precision=2, thousands=',', decimal='.')

,value,change
quarter,,
2023-Q3,"2,433,534,000,000",nan
2023-Q2,"668,547,000,000",-0.73
2023-Q1,"640,178,000,000",-0.04
2022-Q4,"4,306,131,000,000",5.73
2022-Q3,"9,389,198,000,000",1.18
2022-Q2,"11,098,805,000,000",0.18


In [5]:
# 데이터 값의 형식을 지정 (열 이름 대문자로 변환)
df_samsung.style.format_index(str.upper, axis=1)

,VALUE,CHANGE
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459
2022-Q3,9389198000000,1.180426
2022-Q2,11098805000000,0.182082


In [6]:
# 데이터 값의 형식을 지정 (행 인덱스를 소문자로 변환)
df_samsung.style.format_index(str.lower, axis=0)

,value,change
quarter,,
2023-q3,2433534000000,nan
2023-q2,668547000000,-0.725277
2023-q1,640178000000,-0.042434
2022-q4,4306131000000,5.726459
2022-q3,9389198000000,1.180426
2022-q2,11098805000000,0.182082


In [7]:
# 데이터 값의 형식을 지정 (열 이름을 재지정)
df_samsung.style.relabel_index(['영업이익', '영업이익증가율'], axis=1)

,영업이익,영업이익증가율
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459
2022-Q3,9389198000000,1.180426
2022-Q2,11098805000000,0.182082


In [8]:
# 데이터 값의 형식을 지정 (행 인덱스를 재지정)
new_dates = ['2023.09', '2023.06', '2023.03', '2022.12', '2022.09', '2022.06']
df_samsung.style.relabel_index(new_dates, axis=0)

,value,change
quarter,,
2023.09,2433534000000,nan
2023.06,668547000000,-0.725277
2023.03,640178000000,-0.042434
2022.12,4306131000000,5.726459
2022.09,9389198000000,1.180426
2022.06,11098805000000,0.182082


In [9]:
# 연결하기 (concat)
data_styler = df_samsung.style.format(precision=2, thousands=',', decimal='.')
stat_styler = df_samsung.describe().style.format(precision=2, thousands=',', decimal='.')

data_styler.concat(stat_styler)

,value,change
quarter,,
2023-Q3,"2,433,534,000,000",nan
2023-Q2,"668,547,000,000",-0.73
2023-Q1,"640,178,000,000",-0.04
2022-Q4,"4,306,131,000,000",5.73
2022-Q3,"9,389,198,000,000",1.18
2022-Q2,"11,098,805,000,000",0.18
count,6.00,5.00
mean,"4,756,065,500,000.00",1.26
std,"4,493,132,362,855.51",2.59


In [10]:
# Styler 객체를 통해 원본 DataFrame 얻기
df_stat_original = stat_styler.data
df_stat_original

,value,change
count,6.000000e+00,5.000000
mean,4.756066e+12,1.264251
std,4.493132e+12,2.586265
min,6.401780e+11,-0.725277
25%,1.109794e+12,-0.042434
50%,3.369832e+12,0.182082
75%,8.118431e+12,1.180426
max,1.109880e+13,5.726459


In [11]:
# 데이터 숨기기 (열 제외)
df_samsung.style.hide(['value'], axis=1)

,change
quarter,
2023-Q3,nan
2023-Q2,-0.725277
2023-Q1,-0.042434
2022-Q4,5.726459
2022-Q3,1.180426
2022-Q2,0.182082


In [12]:
# 데이터 숨기기 (행 제외)
df_samsung.style.hide(['2023-Q3', '2022-Q3'], axis=0)

,value,change
quarter,,
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459
2022-Q2,11098805000000,0.182082


In [13]:
# 조건부 포맷팅 (각 열의 최대값)
df_samsung.style.highlight_max(color='lightgreen')

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459
2022-Q3,9389198000000,1.180426
2022-Q2,11098805000000,0.182082


In [14]:
# 조건부 포맷팅 (각 열의 최소값)
df_samsung.style.highlight_min(color='orange')

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459
2022-Q3,9389198000000,1.180426
2022-Q2,11098805000000,0.182082


In [15]:
# 조건부 포맷팅 (각 행의 최솟값)
df_samsung.style.highlight_min(color='lightblue', axis=1)

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459
2022-Q3,9389198000000,1.180426
2022-Q2,11098805000000,0.182082


In [16]:
# 조건부 포맷팅 (누락 데이터)
df_samsung.style.highlight_null(color='red')

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459
2022-Q3,9389198000000,1.180426
2022-Q2,11098805000000,0.182082


In [17]:
# 막대 그래프
df_samsung.style.bar(subset=['change'], color='#d65f5f')

,value,change
quarter,,
2023-Q3,2433534000000,nan
2023-Q2,668547000000,-0.725277
2023-Q1,640178000000,-0.042434
2022-Q4,4306131000000,5.726459
2022-Q3,9389198000000,1.180426
2022-Q2,11098805000000,0.182082


In [18]:
# 삼성전자, SK하이닉스 영업이익증가율 데이터를 정리
df_opm = pd.concat([df_samsung['change'], df_skhynix['change']], axis=1)
df_opm.columns=['samsung', 'skhynix']
df_opm

,samsung,skhynix
quarter,,
2023-Q3,NaN,NaN
2023-Q2,-0.725277,0.608341
2023-Q1,-0.042434,0.180501
2022-Q4,5.726459,-0.442032
2022-Q3,1.180426,-1.872093
2022-Q2,0.182082,1.532448


In [19]:
# 모든 원소에 대해서 각각 스타일 적용 
def show_negative(value, props=''):
    return props if value < 0 else None

style_opm = df_opm.style.map(show_negative, props='color:red;')
style_opm

,samsung,skhynix
quarter,,
2023-Q3,nan,nan
2023-Q2,-0.725277,0.608341
2023-Q1,-0.042434,0.180501
2022-Q4,5.726459,-0.442032
2022-Q3,1.180426,-1.872093
2022-Q2,0.182082,1.532448


In [20]:
def highlight_min(style, props=''):
    return np.where(style == np.nanmin(style.values), props, '')

# 데이터프레임 전체 중에서 최소값
style_opm_df = df_opm.style.apply(highlight_min, 
                                   props='color:white;background-color:darkred', 
                                   axis=None)
style_opm_df

,samsung,skhynix
quarter,,
2023-Q3,nan,nan
2023-Q2,-0.725277,0.608341
2023-Q1,-0.042434,0.180501
2022-Q4,5.726459,-0.442032
2022-Q3,1.180426,-1.872093
2022-Q2,0.182082,1.532448


In [21]:
# 데이터프레임 각 열의 최소값
style_opm_col = df_opm.style.apply(highlight_min, 
                                   props='color:white;background-color:darkred', 
                                   axis=0)
style_opm_col

,samsung,skhynix
quarter,,
2023-Q3,nan,nan
2023-Q2,-0.725277,0.608341
2023-Q1,-0.042434,0.180501
2022-Q4,5.726459,-0.442032
2022-Q3,1.180426,-1.872093
2022-Q2,0.182082,1.532448


In [22]:
# 데이터프레임 각 행의 최소값
style_opm_row = df_opm.style.apply(highlight_min, 
                                   props='color:white;background-color:darkred', 
                                   axis=1)
style_opm_row

C:\Users\tsdata\AppData\Local\Temp\ipykernel_32664\4075394969.py:2: RuntimeWarning: All-NaN slice encountered
  return np.where(style == np.nanmin(style.values), props, '')


,samsung,skhynix
quarter,,
2023-Q3,nan,nan
2023-Q2,-0.725277,0.608341
2023-Q1,-0.042434,0.180501
2022-Q4,5.726459,-0.442032
2022-Q3,1.180426,-1.872093
2022-Q2,0.182082,1.532448


In [23]:
# 조건에 따라 다르게 적용

def profit_condition(value):
    if value < 0:
        return "Loss"
    elif value> 0:
        return "Gain"
    else:
        return "No"

def apply_condition(styler):
    styler.set_caption("Gain/Loss")
    styler = styler.format(profit_condition, subset=['change'])
    styler = styler.background_gradient(subset=['change'], axis=None, 
                                        vmin=-10, vmax=10, cmap="RdYlBu_r")
    return styler

df_samsung.style.pipe(apply_condition)

,value,change
quarter,,
2023-Q3,2433534000000,No
2023-Q2,668547000000,Loss
2023-Q1,640178000000,Loss
2022-Q4,4306131000000,Gain
2022-Q3,9389198000000,Gain
2022-Q2,11098805000000,Gain


In [24]:
# HTML CSS 스타일링
cell_hover = {  
    'selector': 'tr:hover',
    'props': [('background-color', '#ffffb3')]
}

stat_styler.set_table_styles([cell_hover])

,value,change
count,6.00,5.00
mean,"4,756,065,500,000.00",1.26
std,"4,493,132,362,855.51",2.59
min,"640,178,000,000.00",-0.73
25%,"1,109,793,750,000.00",-0.04
50%,"3,369,832,500,000.00",0.18
75%,"8,118,431,250,000.00",1.18
max,"11,098,805,000,000.00",5.73
